In [127]:
import pandas as pd
import numpy as np
from io import BytesIO
import requests
import re
from scipy.stats import ttest_ind

# get the rank change for every state for every year

In [27]:
gids = [1938530427, 235986693, 1425149017, 478287251, 1412914561, 0]

In [29]:
def get_df(gid):
    r = requests.get('https://docs.google.com/spreadsheet/ccc?key=1A3ZCPoAZaFWvKailJP3OqL_qI4c6goH9MI9O9HPEago&gid={}&output=csv'.format(gid))
    data = r.content
    return pd.read_csv(BytesIO(data))



In [50]:
df = pd.DataFrame([], index=range(50))

In [51]:
for i in range(len(gids)):
    df[2013+i] = get_df(gids[i]).iloc[:,0]

In [53]:
df.head()

,2013,2014,2015,2016,2017,2018
0,1. NORTH DAKOTA,1. Alaska,1. Hawaii[+1],1. Hawaii,1. South Dakota,Hawaii
1,2. SOUTH DAKOTA,2. Hawaii,2. Alaska [–1],2. Alaska,2. Vermont,Wyoming
2,3. NEBRASKA,3. South Dakota,3. Montana[+2],3. South Dakota,3. Hawaii,Alaska
3,4. MINNESOTA,4. Wyoming,4. Colorado[+2],4. Maine,4. Minnesota,Montana
4,5. MONTANA,5. Montana,5. Wyoming [–1],5. Colorado,5. North Dakota,Utah


# for each state, for each year, get its rank for that year

In [70]:
def clean_name(x):
    x = re.sub('^\s*\d+\.\s*', '', x)
    x = re.sub('\s*\[.+\]\s*$', '', x)
    x = x.lower().strip()
    return x

In [71]:
df = df.applymap(clean_name)

In [75]:
states = df.stack().unique()

In [76]:
len(states)

50

In [92]:
df_ = pd.DataFrame([], index=states, columns=df.columns)

In [93]:
for i in range(len(gids)):
    for state in states:
        df_.loc[state, 2013+i] = df.index[df[2013+i]==state][0]

In [98]:
df_.head()

,2013,2014,2015,2016,2017,2018
north dakota,0,22,14,15,4,9
alaska,15,0,1,1,24,2
hawaii,7,1,0,0,2,0
south dakota,1,2,5,2,0,8
vermont,5,12,28,5,1,6


In [118]:
df_diff = df_.T.diff().dropna()

In [119]:
df_diff

,north dakota,alaska,hawaii,south dakota,vermont,wyoming,nebraska,montana,minnesota,colorado,...,michigan,missouri,ohio,oklahoma,indiana,louisiana,mississippi,arkansas,kentucky,west virginia
2014,22.0,-15.0,-6.0,1.0,7.0,-30.0,4.0,0.0,7.0,-1.0,...,5.0,-2.0,1.0,-3.0,8.0,-1.0,-2.0,-2.0,0.0,0.0
2015,-8.0,1.0,-1.0,3.0,16.0,1.0,13.0,-2.0,-4.0,-2.0,...,-3.0,4.0,0.0,9.0,-2.0,2.0,-3.0,1.0,0.0,0.0
2016,1.0,0.0,0.0,-3.0,-23.0,7.0,-5.0,5.0,2.0,1.0,...,0.0,-9.0,-2.0,0.0,1.0,1.0,-2.0,2.0,0.0,0.0
2017,-11.0,23.0,2.0,-2.0,-4.0,4.0,2.0,2.0,-5.0,1.0,...,-7.0,3.0,-1.0,-2.0,-5.0,6.0,6.0,2.0,-4.0,0.0
2018,5.0,-22.0,-2.0,8.0,5.0,-14.0,1.0,-6.0,8.0,0.0,...,1.0,1.0,-6.0,-1.0,-1.0,-6.0,0.0,1.0,3.0,0.0


# get the rank change for legalization years per state

In [136]:
# https://en.wikipedia.org/wiki/Timeline_of_cannabis_laws_in_the_United_States

In [131]:
legalization = {
#     "Colorado": 2012,
#     "Washington": 2012,
    "Alaska":2014,
    "Oregon":2014,
    "California":2016, "Nevada":2016, "Maine":2016, "Massachusetts":2016,
#     "Vermont":2018,
#     "Michigan":2018,
#     "Illinois":2019
}

In [113]:
df_diff[next(iter(legalization.keys())).lower()][2014]

-1.0

In [132]:
legalization_diff = [df_diff[k.lower()][v+1] for k,v in legalization.items()]

# check whether legalization shows significant increase

In [133]:
ttest_ind(list(df_diff.stack()), legalization_diff)

Ttest_indResult(statistic=-1.5365941037653152, pvalue=0.125637461537519)

In [135]:
legalization_diff

[1.0, 4.0, 1.0, 9.0, 20.0, -3.0]

In [134]:
np.mean(legalization_diff)

5.333333333333333

In [129]:
df_diff.stack().mean()

0.0